In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool

In [ ]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon_item_mapping=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [ ]:
df.head()

In [ ]:
def get_coupons_for_customer(cust_id):
    return df[df.customer_id==cust_id].coupon_id.unique().tolist()
def get_items_for_coupons(coupon_list):
    return coupon_item_mapping[coupon_item_mapping.coupon_id.isin(coupon_list)].item_id.unique().tolist()
def get_custs_for_coupon(coup_id):
    return df[df.coupon_id==coup_id].customer_id.unique().tolist()
def get_coupons_for_customer_camp(cust_id,camp_id):
    return df[(df.customer_id==cust_id)&(df.campaign_id==camp_id)].coupon_id.unique().tolist()

In [ ]:
data_=df.iloc[0]

In [ ]:
import datetime

In [ ]:
df.head()

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    #### this is for test_feat_2
    l=get_custs_for_coupon(data_.coupon_id)
    l.remove(data_.customer_id)
    temp=transaction[(transaction.customer_id.isin(l))&\
                     transaction.item_id.isin(get_items_for_coupons([data_.coupon_id]))
                    ]
    
    
    if temp.shape[0]>0:
            feats=[data_.id]+[sum(temp.coupon_discount<0),temp.shape[0],temp.coupon_discount.sum()]
    else:
        feats=[data_['id']]+np.repeat(0,3).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, df.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['id','test1_var2','test2_var2','test3_var2']
test_feat=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
test_feat.to_csv('../test_feat_2.csv',index=False)

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    #### this is for test_feat_2
    temp=transaction[(transaction.customer_id!=data_.customer_id)&\
                     transaction.item_id.isin(get_items_for_coupons([data_.coupon_id]))
                    ]
    
    if temp.shape[0]>0:
            feats=[data_.id]+[sum(temp.coupon_discount<0),temp.shape[0],temp.coupon_discount.sum()]
    else:
        feats=[data_['id']]+np.repeat(0,3).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, df.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['id','test1','test2','test3']
test_feat=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
test_feat.to_csv('../test_feat.csv',index=False)

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id.isin(get_custs_for_coupon(data_.coupon_id)))&\
                (transaction.item_id.isin(get_items_for_coupons([data_.coupon_id])))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['coupon_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['coupon_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['coupon_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','coupon_id','num_bought_coup_var1','num_discount_availed_coup_var1','frac_discount_availed_coup_var1','sum_discount_coup_var1',
'selling_price_sum_coup_var1',
'coupon_discount_mean_coup_var1','selling_price_mean_coup_var1','other_discount_sum_coup_var1',
      'other_discount_mean_coup_var1','cd_sp_ratio_mean_coup_var1','cd_sp_ratio_sum_coup_var1'
     ]
coup_hist1=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
coup_hist1.to_csv('../coup_hist1_v2.csv',index=False)

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_.customer_id)&\
                (transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer(data_.customer_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var1','num_discount_availed_cust_var1','frac_discount_availed_cust_var1','sum_discount_cust_var1',
'selling_price_sum_cust_var1',
'coupon_discount_mean_cust_var1','selling_price_mean_cust_var1','other_discount_sum_cust_var1',
      'other_discount_mean_cust_var1','cd_sp_ratio_mean_cust_var1','cd_sp_ratio_sum_cust_var1'
     ]
cust_coup_hist=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
cust_coup_hist.head()

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_.customer_id)&\
    (transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer_camp(data_.customer_id,data_.campaign_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var3','num_discount_availed_cust_var3','frac_discount_availed_cust_var3','sum_discount_cust_var3',
'selling_price_sum_cust_var3',
'coupon_discount_mean_cust_var3','selling_price_mean_cust_var3','other_discount_sum_cust_var3',
      'other_discount_mean_cust_var3','cd_sp_ratio_mean_cust_var3','cd_sp_ratio_sum_cust_var3'
     ]
cust_coup_hist3=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
cust_coup_hist3.head()

In [ ]:
cust_coup_hist3.to_csv('../cust_coup_hist3_v2.csv',index=False)

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer(data_.customer_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var2','num_discount_availed_cust_var2','frac_discount_availed_cust_var2','sum_discount_cust_var2',
'selling_price_sum_cust_var2',
'coupon_discount_mean_cust_var2','selling_price_mean_cust_var2','other_discount_sum_cust_var2',
      'other_discount_mean_cust_var2','cd_sp_ratio_mean_cust_var2','cd_sp_ratio_sum_cust_var2'
     ]
cust_coup_hist2=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
cust_coup_hist2.head()

In [ ]:
cust_coup_hist.to_csv('../customer_coupon_var1_v2.csv',index=False)
cust_coup_hist2.to_csv('../customer_coupon_var2_v2.csv',index=False)